In [1]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver

# (201105) 추가한 모듈, BeautifulSoup4 사용 추가
from bs4 import BeautifulSoup

# lxml
from lxml import html

# pandas
import pandas as pd

# requests, json, re
import requests
import json

In [2]:
# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

# 샘플 키워드 리스트
animal_list = ['희귀 동물', '동물농장', '위험한 동물']
music_list = ['아이돌 노래','음악 연주회', '추억의 음악']
game_list = ['신작 게임','게임 매드무비','블리자드', '넷마블']
politics_list = ['오늘의 날씨', '교통 뉴스', '행복한 뉴스']
vlog_list = ['코로나vlog', '일상 vlog' , '수능생vlog']
travel_list = ['국내 여행지 추천', '카라반', '겨울 여행']
sports_list = ['배구', '컬링', '동계올림픽']
comedy_list = ['개그맨', '90년대 개그', '부장님 개그']
entertainment_list = ['토크쇼', '예능 하이라이트']
movie_list = ['시사회', '12월 개봉영화', '90년대 영화']
education_list = ['유아교육', '초등교육', '중학교교육']
science_list = ['4차산업혁명', 'IOT', '과학 실험']

# 모든 키워드를 다 합친 리스트
all_search_keyword_list = sports_list + movie_list + vlog_list + music_list + comedy_list + game_list + politics_list + travel_list + animal_list + entertainment_list + education_list + science_list
#all_search_keyword_list = science_list
print("All keywords list: ", all_search_keyword_list)

# video_id_list
# sports_video_id_list = []
# movie_video_id_list = []
# kids_video_id_list = []
# music_video_id_list = []
# comedy_video_id_list = []
# game_video_id_list = []
# politics_video_id_list = []
# cook_video_id_list = []
# animal_video_id_list = []

# 샘플 키워드 딕셔너리
# search_dict = {
#     'Sports' : [sports_list, sports_video_id_list]
#     'Movie' : [movie_list, movie_video_id_list]
#     'Kids' : [kids_list, kids_video_id_list]
#     'Music' : [music_list, music_video_id_list]
#     'Comedy' : [comedy_list, comedy_video_id_list]
#     'Game' : [game_list, game_video_id_list]
#     'Politics' : [politics_list, politics_video_id_list]
#     'Cook' : [cook_list, cook_video_id_list]
#     'Animal' : [animal_list, animal_video_id_list]
# }

All keywords list:  ['배구', '컬링', '동계올림픽', '시사회', '12월 개봉영화', '90년대 영화', '코로나vlog', '일상 vlog', '수능생vlog', '아이돌 노래', '음악 연주회', '추억의 음악', '개그맨', '90년대 개그', '부장님 개그', '신작 게임', '게임 매드무비', '블리자드', '넷마블', '오늘의 날씨', '교통 뉴스', '행복한 뉴스', '국내 여행지 추천', '카라반', '겨울 여행', '희귀 동물', '동물농장', '위험한 동물', '토크쇼', '예능 하이라이트', '유아교육', '초등교육', '중학교교육', '4차산업혁명', 'IOT', '과학 실험']


In [3]:
# 샘플 키워드 리스트 #2

In [4]:
# 검색어 결과별로 비디오 id를 모아주는 함수 정의
# 리턴은 dictionary
def get_search_keyword_scrap_video_id(search_keyword_list):
    # 최종적으로 리턴할 dictionary 정의
    youtube_id_dict = {}

    for word in search_keyword_list:
        # youtube_id_list
        youtube_id_list = []
        
        # webdriver define
        driver_path = './chromedriver.exe'
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(30) # or bigger second
        
        # 열고자 하는 search url 정의
        url = "https://www.youtube.com/results?search_query=" + word
        print("full url: ", url)

        # 해당 url로 창 열기
        driver.get(url)
        time.sleep(5)

        # 첫 동영상 시작 시 시작번호(index = 1)
        index = 1
        SCROLL_PAUSE_TIME = 2

        # infinite scroll
        while True:
            try:
                youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer['+str(index)+']/div[1]/div/div[1]/div/h3/a'

                # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
                if index % 4 == 0:
                    # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    last_height = driver.execute_script("return document.body.scrollHeight")

                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(SCROLL_PAUSE_TIME)

                        new_height = driver.execute_script("return document.body.scrollHeight")

                        if new_height == last_height:
                            break

                        last_height = new_height


                youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
                youtube_id = youtube_title_link.get_attribute('href')
                youtube_id = youtube_id.split("v=")[1].split("&")[0]
                youtube_id_list.append(youtube_id)

                # 리스트에 들어간 데이터 중간 확인
                print("index_no: ", index)
                print("youtube_id: ", youtube_id)

                index += 1

            except Exception as e:
                print()
                print("Exception: ", e)
                break
        # dictionary에 검색 키워드 별 리스트 추가
        youtube_id_dict[word] = youtube_id_list
        # youtube_id 수집이 끝난 브라우저 닫기        
        driver.close()
    
    return youtube_id_dict

In [5]:
youtube_id_dict = get_search_keyword_scrap_video_id(all_search_keyword_list)
print("youtube_id_dict: ", youtube_id_dict)

full url:  https://www.youtube.com/results?search_query=배구
index_no:  1
youtube_id:  p-VbWwxEsEY
index_no:  2
youtube_id:  zGEDyMoJldY
index_no:  3
youtube_id:  UnjEQajfgww
index_no:  4
youtube_id:  XE9WwgCZVic
index_no:  5
youtube_id:  DfszN2M_GX4
index_no:  6
youtube_id:  ToLm5uFuoD8
index_no:  7
youtube_id:  VKsdn2F2X9I
index_no:  8
youtube_id:  8AhwVIUlNXo
index_no:  9
youtube_id:  JWYz0wkAPhI
index_no:  10
youtube_id:  Szozl8dYPR8
index_no:  11
youtube_id:  PWHpeQGi7fw
index_no:  12
youtube_id:  GgoVTbxyTbI
index_no:  13
youtube_id:  -aEBhJEOof0
index_no:  14
youtube_id:  l3YrnTbmTb8
index_no:  15
youtube_id:  xV-MRREHT6w
index_no:  16
youtube_id:  90sQ-zsV9uo
index_no:  17
youtube_id:  MjtSzNpOtsE
index_no:  18
youtube_id:  tLxa7sHaxRo
index_no:  19
youtube_id:  Cn4SEo4dDpI
index_no:  20
youtube_id:  qFwxHryTTrc

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-se

full url:  https://www.youtube.com/results?search_query=일상 vlog
index_no:  1
youtube_id:  8q9nbJ9usXU
index_no:  2
youtube_id:  0LA8wMdWmh8
index_no:  3
youtube_id:  Rodo4zWMuBU
index_no:  4
youtube_id:  MSLDIQI9lwI
index_no:  5
youtube_id:  NwmcYoVFvIw
index_no:  6
youtube_id:  -afvKSQjCXA
index_no:  7
youtube_id:  _ir9WyK02fo
index_no:  8
youtube_id:  k_XdVeEAagM
index_no:  9
youtube_id:  FBmwwPH95H0
index_no:  10
youtube_id:  CcTCHTAHKSA
index_no:  11
youtube_id:  EX5Ow6nhT_E
index_no:  12
youtube_id:  jK2YEJS74vc
index_no:  13
youtube_id:  Q0UveVxohnM
index_no:  14
youtube_id:  Shwhh3x-3g8
index_no:  15
youtube_id:  d3PL4_1ohoo
index_no:  16
youtube_id:  BKOs2uTWFFk
index_no:  17
youtube_id:  gRRMv85oKmk
index_no:  18
youtube_id:  6ppMQ2q6aPE
index_no:  19
youtube_id:  4SzWSVB-y1U
index_no:  20
youtube_id:  QtzESd_e6HY

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-colu

index_no:  4
youtube_id:  iHEr4I6xClg
index_no:  5
youtube_id:  b8uUHiGecUM
index_no:  6
youtube_id:  POxCDH-bjRM
index_no:  7
youtube_id:  AaRf0gjJf-s
index_no:  8
youtube_id:  -JA4jE0cOlA
index_no:  9
youtube_id:  vTuFuzg3eEg
index_no:  10
youtube_id:  VooagiGc_lQ
index_no:  11
youtube_id:  rzQ3Y1B1Opw
index_no:  12
youtube_id:  bWf6wJf5KNA
index_no:  13
youtube_id:  GWxnqLt5dYU
index_no:  14
youtube_id:  fhGu_t9VYEU
index_no:  15
youtube_id:  CG9MqmmOJqo
index_no:  16
youtube_id:  2snJWLg6krw
index_no:  17
youtube_id:  GgKIIda-JEY
index_no:  18
youtube_id:  5xnTD_0kS8I
index_no:  19
youtube_id:  7nIvVAc79Zc

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[20]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.you

index_no:  8
youtube_id:  NiPfs0kFlUU
index_no:  9
youtube_id:  8zFUwVfz3L4
index_no:  10
youtube_id:  HHS6pmFaC58
index_no:  11
youtube_id:  jsQtdMV6O_4
index_no:  12
youtube_id:  giHe9YpqxAc
index_no:  13
youtube_id:  8hweo4sdcdg
index_no:  14
youtube_id:  6uRFEVuzdjk
index_no:  15
youtube_id:  pv9zzx_Xa7I
index_no:  16
youtube_id:  KOyxNMZcwi0
index_no:  17
youtube_id:  rYdboSisJPY
index_no:  18
youtube_id:  tmF38mU1elk
index_no:  19
youtube_id:  hFXL8QKbH68
index_no:  20
youtube_id:  2ziPUfQ2Arw

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=국내 여행지 추천
index_no:  1
youtube_id:  IHZci7auKwU
index_no:  2
youtube_id:  _gffY_2kn

index_no:  8
youtube_id:  Z6zR87zViMc
index_no:  9
youtube_id:  Rm6ppyyGOZU
index_no:  10
youtube_id:  y7zZNLm2C9I
index_no:  11
youtube_id:  UmeHmlfJR2M
index_no:  12
youtube_id:  DmT-hlCZwLU
index_no:  13
youtube_id:  9ehSG7OpbcQ
index_no:  14
youtube_id:  0Vrv6hWoko0
index_no:  15
youtube_id:  dquhEALQTNo
index_no:  16
youtube_id:  q_fzpqdeP8I
index_no:  17
youtube_id:  rG9Ux4fnPT8
index_no:  18
youtube_id:  EAAhSL59ddY
index_no:  19
youtube_id:  Slsch9PT_xs
index_no:  20
youtube_id:  yvQMsYInc28

Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer[21]/div[1]/div/div[1]/div/h3/a"}
  (Session info: chrome=87.0.4280.88)

full url:  https://www.youtube.com/results?search_query=예능 하이라이트
index_no:  1
youtube_id:  B5MCLqaNftI
index_no:  2
youtube_id:  7tq0jEtduj

In [6]:
# 특정 검색어 키워드 기반으로 크롤링 해 주는 크롤러 정의
def scrap_search_detail_video_data(video_id_dict):
    # 모든 결과를 누적해서 보여주는 df 정의
    youtube_raw_data_columns = ['title', 'authorNick','urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt',
                                'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly',
                                'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords', 'searchKeyword']
    
    # 모든 DF를 누적할 DataFrame정의
    total_df = pd.DataFrame(columns = youtube_raw_data_columns)
    
    # 각 video_id에 해당하는 유튜브 영상 별로 상세 데이터 수집
    for keyword, youtube_id_list in video_id_dict.items():
        
        print("==========outer loop==========")
        print("keyword: ", keyword)
        print("video_id_list: ", youtube_id_list)
        
        # 수집할 데이터의 리스트
        title_list = []
        author_list = []
        #description_list = []
        url_list = []
        thumb_url_list = []
        like_cnt_list = []
        dislike_cnt_list = []
        follow_cnt_list = []
        view_cnt_list = []
        duration_list = []
        youtube_category_list = []
        youtube_category_sn_list = []
        is_family_friendly_list = []
        title_hash_tags_list = []
        description_hash_tags_list = []
        video_id_list = [] 
        author_id_list = []
        yupload_time_list = []
        keywords_list = []
        searchKeyword_list = []
        
        # youtube_raw_data를 받을 dictionary 정의
        youtube_raw_data = {}
        
        for video_id in youtube_id_list:
            print("==========inner loop==========")
            
            # webdriver 초기화
            driver_path = './chromedriver.exe'
            driver = webdriver.Chrome(driver_path)
            driver.implicitly_wait(100) # or bigger second

            # origin_url 정의
            origin_url = 'https://www.youtube.com/watch?v='

            # 넣어주는 url 값
            url = origin_url + video_id

            # scriptTag에서 긁어지는 데이터
            script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

            # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'lxml')

            # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
            yt_initial_data_xpath = '/html/body/script[18]'
            
            driver.get(url)
            time.sleep(5)
            
            # 데이터를 담을 변수 정의(scriptTag, metadata, ytInitialData)
            try:
                # Data 크롤링
                # 각 유형별로 다른 로직으로 데이터를 크롤링한다
                # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
                script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

                script_tag_json = json.loads(script_tag_str)

                time.sleep(5)

                # print("script_tag_json: ", script_tag_json)
                # print("script_tag_json_type: ", type(script_tag_json))

                # Data result test
                title = script_tag_json['name']
                author = script_tag_json['author']
                #description = script_tag_json['description']
                thumb_url = script_tag_json['thumbnailUrl'][0]
                view_cnt = int(script_tag_json['interactionCount'])
                duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
                youtube_category = script_tag_json['genre']
                # addition : youtube_category_sn
                youtube_category_sn = youtube_category_dict[youtube_category]

                print()
                print("=" * 20, "Youtube Raw Data Descriptions", "=" * 20)
                print("title: ", title)
                print("author: ", author)
                #print("description: ", description)
                print("thumb_url: ", thumb_url)
                print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
                print("duration: ", duration, " type: ", type(duration))
                print("youtube_category: ", youtube_category)
                print("youtube_category_id: ", youtube_category_sn)

                # meta tag data crawling
                is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
                author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
                yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
                keywords = ",".join([tag['content'].replace("#", "") for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

                print("is_family_friendly: ", is_family_friendly)
                print("video_id: ", video_id)
                print("author_id: ", author_id)
                print("yupload_time: ", yupload_time)
                print("keywords: ", keywords)

                # ytInitialData crawling
                yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
                yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('var ytInitialData =', '').strip()
                # print("yt_initial_data: ", yt_initial_data_str)

                time.sleep(5)

                # data to json
                # json loading이 간헐적으로 안되는 현상 수정 필요
                yt_initial_data_json = json.loads(yt_initial_data_str)

                # print("yt_initial_data_json: ", yt_initial_data_json)

                # video_info_renderer
                video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
                video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
                video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

                # print("video_primary_info_renderer: ", video_primary_info_renderer)
                # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

                # 좋아요 & 싫어요
                like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

                print("like_dislike_cnt: ", like_dislike_cnt)

                like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
                dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))

                print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
                print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

                # 제목 타이틀 해시태그
                # 타이틀 해시태그가 없는 경우 빈문자열로 처리
                raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
                if raw_title_hash_tags != "":
                    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
                    title_hash_tag_list = []
                    for tag in title_hash_tags:
                        if tag['text'] not in [" ", ""]:
                            title_hash_tag_list.append(tag['text'][1:])
                            # print("title_hash_tag: ", tag['text'][1:])
                    title_hash_tags_str = ",".join(title_hash_tag_list)
                else:
                    title_hash_tags_str = ""
                print("title_hash_tags_str: ", title_hash_tags_str)

                # 내용 해시태그
                description_hash_tags = video_secondary_info_renderer['description']['runs']
                description_hash_tag_list = []
                for tag in description_hash_tags:
                    if tag['text'] not in [" ", ""]:
                        if "#" in tag['text']:
                            description_hash_tag_list.append(tag['text'][1:])
                            # print("description_hash_tag: ", tag['text'][1:])

                description_hash_tags_str = ",".join(description_hash_tag_list)
                print("description_hash_tags_str: ", description_hash_tags_str)

                # 구독자 수
                # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
                raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
                if raw_follow != "":
                    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
                    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
                    print("follow_cnt: ", follow_cnt)

                    # 구독자 수 수치화 필터링
                    num_count_dict = {
                        '천' : 1000,
                        '만' : 10000,
                        '억' : 100000000,
                        '조' : 1000000000000
                    }

                    # print("follow_cnt: ", follow_cnt)
                    # print("follow_num_cnt: ", follow_cnt[-1])
                    # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
                    if follow_cnt[-1] in num_count_dict.keys():
                        follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
                    else:
                        follow_cnt_num = int(follow_cnt)
                else:
                    follow_cnt_num = 0

                print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))

                # 예외처리에 걸리지 않고 무사히 넘어왔다면 list에 데이터가 담긴다
                # scriptTag data appending
                title_list.append(title)
                author_list.append(author)
                #description_list.append(description)
                url_list.append(url)
                thumb_url_list.append(thumb_url)
                view_cnt_list.append(view_cnt)
                duration_list.append(duration)
                youtube_category_list.append(youtube_category)
                youtube_category_sn_list.append(youtube_category_sn)

                # meta tag data appending
                is_family_friendly_list.append(is_family_friendly)
                video_id_list.append(video_id)
                author_id_list.append(author_id)
                yupload_time_list.append(yupload_time)
                keywords_list.append(keywords)
                searchKeyword_list.append(keyword)

                # yInitialData data appending
                like_cnt_list.append(like_cnt)
                dislike_cnt_list.append(dislike_cnt)
                title_hash_tags_list.append(title_hash_tags_str)
                description_hash_tags_list.append(description_hash_tags_str)
                follow_cnt_list.append(follow_cnt_num)
                
            except Exception as e:
                print()
                print("Exception: ", e)
                driver.close()
                continue
            
            print("=" * 20, "Youtube Raw Data Descriptions Ended", "=" * 20, "\n")
            
            driver.close()

        # 얻어온 데이터를 dictionary에 저장
        youtube_raw_data['title'] = title_list
        youtube_raw_data['authorNick'] = author_list
        #youtube_raw_data['description'] = description_list
        youtube_raw_data['urlPath'] = url_list
        youtube_raw_data['thumbUrl'] = thumb_url_list
        youtube_raw_data['likeCnt'] = like_cnt_list
        youtube_raw_data['dislikeCnt'] = dislike_cnt_list
        youtube_raw_data['followCnt'] = follow_cnt_list
        youtube_raw_data['viewCnt'] = view_cnt_list
        youtube_raw_data['duration'] = duration_list
        youtube_raw_data['utbCategory'] = youtube_category_list
        youtube_raw_data['utbCateSN'] = youtube_category_sn_list
        youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
        youtube_raw_data['titleHashTags'] = title_hash_tags_list
        youtube_raw_data['descHashTags'] = description_hash_tags_list
        youtube_raw_data['utbVideoID'] = video_id_list
        youtube_raw_data['authorID'] = author_id_list
        youtube_raw_data['yUploadDateTime'] = yupload_time_list
        youtube_raw_data['keywords'] = keywords_list
        youtube_raw_data['searchKeyword'] = searchKeyword_list
        
        print("youtube_raw_data: ", youtube_raw_data)
        
        # DataFrame 생성
        df = pd.DataFrame(youtube_raw_data)
        
        total_df.append(df, ignore_index = True)

        # 모아둔 DataFrame을 csv로 변환
        df.to_csv("./1210_youtube_.csv", sep="|", mode="a",header= False,index=False, encoding='utf-8-sig')

    return total_df

In [7]:
scrap_search_detail_video_data(youtube_id_dict)

==========outer loop==========
keyword:  배구
video_id_list:  ['p-VbWwxEsEY', 'zGEDyMoJldY', 'UnjEQajfgww', 'XE9WwgCZVic', 'DfszN2M_GX4', 'ToLm5uFuoD8', 'VKsdn2F2X9I', '8AhwVIUlNXo', 'JWYz0wkAPhI', 'Szozl8dYPR8', 'PWHpeQGi7fw', 'GgoVTbxyTbI', '-aEBhJEOof0', 'l3YrnTbmTb8', 'xV-MRREHT6w', '90sQ-zsV9uo', 'MjtSzNpOtsE', 'tLxa7sHaxRo', 'Cn4SEo4dDpI', 'qFwxHryTTrc']
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [ V리그 ] 20-21시즌 여자배구 흥국생명 VS GS칼텍스 (20-12-05)
author:  Damn Dumpster
thumb_url:  https://i.ytimg.com/vi/p-VbWwxEsEY/hqdefault.jpg
view_cnt:  7475  type:  <class 'int'>
duration:  9825  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  p-VbWwxEsEY
author_id:  UCngxttoOfkvPgpSumtqUhSg
yupload_time:  2020-12-05
keywords:  V리그 20-21시즌,여자배구,흥국생명,GS칼텍스,V리그,배구,여자프로배구,프로배구,이주아,이재영,톰시아,lee jaeyeong,lee juah,tomsia,berenika tomsia,volleyball,korean volleyball,korean league 


Exception:  Unterminated string starting at: line 1 column 17326 (char 17325)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  여자배구 세터 없이 경기하면 벌어지는 끔찍한 일, V리그 희귀한 명장면 모음
author:  배구왕 TV
thumb_url:  https://i.ytimg.com/vi/PWHpeQGi7fw/maxresdefault.jpg
view_cnt:  116315  type:  <class 'int'>
duration:  379  type:  <class 'int'>
youtube_category:  Sports
youtube_category_id:  17
is_family_friendly:  true
video_id:  PWHpeQGi7fw
author_id:  UCvynsXQSUImxMIRFFkzgGCg
yupload_time:  2020-12-02
keywords:  고예림,현대건설배구단,이다영,흥국생명배구단,여자배구 범실
like_dislike_cnt:  259 / 13
like_cnt:  259  type:  <class 'int'>
dislike_cnt:  13  type:  <class 'int'>
title_hash_tags_str:  여자배구,배구,여자배구선수
description_hash_tags_str:  여자배구,배구,여자배구선수
follow_cnt:  1.56만
follow_cnt_num:  15600  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Desc


==================== Youtube Raw Data Descriptions ====================
title:  여자 컬링 준결승 한일전 주요장면..1엔드부터 11엔드까지 (하이라이트) / SBS / 2018 평창올림픽
author:  SBS 뉴스
thumb_url:  https://i.ytimg.com/vi/9LkGvRNS3WE/hqdefault.jpg
view_cnt:  1755184  type:  <class 'int'>
duration:  507  type:  <class 'int'>
youtube_category:  News & Politics
youtube_category_id:  25
is_family_friendly:  true
video_id:  9LkGvRNS3WE
author_id:  UCkinYTS9IHqOEwR1Sze2JTw
yupload_time:  2018-02-23
keywords:  뉴스,NEWS,news,SBS,sbs,SBS NEWS,SBS8NEWS,SBS뉴스,SBS8뉴스,sbs뉴스,sbs8뉴스,에스비에스,에스비에스뉴스,서울방송,SBS보도국,SEOUL BROADCASTING SYSTEM,seoul broadcasting system,sbs 뉴스,korea,KOREA,한국,대한민국,한국 뉴스,korea news,korean news,KOREA NEWS,KOREAN NEWS,8뉴스,컬링,컬링 여자,여자 컬링,한일전,컬링 한일전,컬링 승리,영미,김은정,김영미,김경애,김선영
like_dislike_cnt:  12,568 / 243
like_cnt:  12568  type:  <class 'int'>
dislike_cnt:  243  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  113만
follow_cnt_num:  1130000  type:  <class 'int'>
================


Exception:  'videoPrimaryInfoRenderer'
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  컬링(표적 도전 활동)규칙, 학교 체육 컬링, 온라인 체육, 등교 체육, 도전 활동, 표적 도전
author:  양쌤스쿨Yangssaem.school
thumb_url:  https://i.ytimg.com/vi/x4C2_SoPx58/maxresdefault.jpg
view_cnt:  7975  type:  <class 'int'>
duration:  708  type:  <class 'int'>
youtube_category:  People & Blogs
youtube_category_id:  22
is_family_friendly:  True
video_id:  x4C2_SoPx58
author_id:  UClg5CFhWb8I0rW2JTMFA1Zg
yupload_time:  2020-05-31
keywords:  

Exception:  Unterminated string starting at: line 1 column 72820 (char 72819)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  평창동계올림픽-컬링에 숨은 과학 마찰력!
author:  KTV국민방송
thumb_url:  https://i.ytimg.com/vi/kGywIAZlQ-4/maxresdefault.jpg
view_cnt:  97200  type:  <class 'int'>
duration:  380  type:  <class 'int'>
youtube_category:  News & Politics
youtube_category_id:  25
is_family


==================== Youtube Raw Data Descriptions ====================
title:  2018 평창 동계올림픽 개회식.. 고화질 다시보기 (풀영상) / SBS / 2018 평창올림픽
author:  SBS 뉴스
thumb_url:  https://i.ytimg.com/vi/SIy7odd4BvM/maxresdefault.jpg
view_cnt:  1959956  type:  <class 'int'>
duration:  10813  type:  <class 'int'>
youtube_category:  News & Politics
youtube_category_id:  25
is_family_friendly:  True
video_id:  SIy7odd4BvM
author_id:  UCkinYTS9IHqOEwR1Sze2JTw
yupload_time:  2018-02-11
keywords:  뉴스,NEWS,news,SBS,sbs,SBS8NEWS,SBS뉴스,SBS8뉴스,sbs8뉴스,에스비에스,서울방송,seoul broadcasting system,sbs 뉴스,KOREA,한국,대한민국,한국 뉴스,korea news,KOREA NEWS,KOREAN NEWS,8뉴스,개막식,개회식,평창 개막식,평창개막식,평창개회식,평창 개회식,평창 올림픽 개회식,평창 올림픽 개막식,SBS 개회식,SBS 개막식,sbs 개회식,인면조,김연아 성화,김연아 성화 점화,드론 오륜기,평창 드론

Exception:  Unterminated string starting at: line 1 column 90908 (char 90907)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  [풀영상] 2018 평창동계올림픽 쇼트트랙 여자 3천미터 계주 결승...'눈물의 금메달' 획득 순간 / KBS뉴스(

...


==================== Youtube Raw Data Descriptions ====================
title:  당신이 알아야 할 4차 산업혁명 / YTN 사이언스
author:  YTN 사이언스
thumb_url:  https://i.ytimg.com/vi/6VYascb96cI/maxresdefault.jpg
view_cnt:  396266  type:  <class 'int'>
duration:  2767  type:  <class 'int'>
youtube_category:  Science & Technology
youtube_category_id:  28
is_family_friendly:  true
video_id:  6VYascb96cI
author_id:  UCZdBJIbJz0P9xyFipgOj1fA
yupload_time:  2019-04-04
keywords:  YTN사이언스,프로그램,사이언스프로그램,제4의 물결,초연결시대,4차산업혁명,사물인터넷,서유리,근거리무선통신,5G,웨어러블,디바이스,빅데이터,클라우드,컴퓨팅,인공지능

Exception:  Unterminated string starting at: line 1 column 117296 (char 117295)
==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  4차 산업혁명 - 4차 산업혁명이란 무엇인가?
author:  지식보관소
thumb_url:  https://i.ytimg.com/vi/DTpHyc0MKWc/maxresdefault.jpg
view_cnt:  123626  type:  <class 'int'>
duration:  601  type:  <class 'int'>
youtube_category:  Education
youtube_category_id:  27
is_family_friendly:

like_dislike_cnt:  143 / 11
like_cnt:  143  type:  <class 'int'>
dislike_cnt:  11  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  8.28만
follow_cnt_num:  82800  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  4차 산업혁명, 어떻게 대처할 것인가?_김용삼기자의 왜곡된 근현대사 바로 알기 33 (2020.11.29)
author:  Charity Baptist Church
thumb_url:  https://i.ytimg.com/vi/y6D8C54I8x8/maxresdefault.jpg
view_cnt:  7690  type:  <class 'int'>
duration:  3555  type:  <class 'int'>
youtube_category:  Education
youtube_category_id:  27
is_family_friendly:  true
video_id:  y6D8C54I8x8
author_id:  UC-qEqMeTaSnmy3kdoN-rF4w
yupload_time:  2020-12-01
keywords:  4차 산업혁명 어떻게 대처할것인가,근현대사,김용삼 기자,김용삼기자의 왜곡된 근현대사 바로 알기,김용삼 기자의 왜곡된 근현대사 바로 알기 33

Exception:  'sentimentBar'
==========inner loop==========

==================== Youtube Raw Data D


==================== Youtube Raw Data Descriptions ====================
title:  놀라운 과학 실험 19가지
author:  5분 Tricks 플레이
thumb_url:  https://i.ytimg.com/vi/FmU0guKM7aE/maxresdefault.jpg
view_cnt:  91764  type:  <class 'int'>
duration:  482  type:  <class 'int'>
youtube_category:  Howto & Style
youtube_category_id:  26
is_family_friendly:  true
video_id:  FmU0guKM7aE
author_id:  UCfMXU4AC4DX81ewfnGJYTDg
yupload_time:  2019-10-27
keywords:  트릭,간단한 인생 팁,팁,계란,교육,탄산 음료,유용한 것들,어린이집,학교,diy 액티비티,색칠,diy 프로젝트,활동,홈 스쿨링,과학,어린이,5분 만들기,키즈,어린이 과학,수공예,요령,diy,인생 팁,5분 만들기 키즈,쉬운 인생 팁,레몬,학교 팁,과학 프로젝트,집에서 하는 과학,과학 실험,학습,물,취미,유치원,스스로 하기,집에서 과학,공예
like_dislike_cnt:  441 / 33
like_cnt:  441  type:  <class 'int'>
dislike_cnt:  33  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  36.7만
follow_cnt_num:  367000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Dat

like_dislike_cnt:  233 / 7
like_cnt:  233  type:  <class 'int'>
dislike_cnt:  7  type:  <class 'int'>
title_hash_tags_str:  
description_hash_tags_str:  
follow_cnt:  144만
follow_cnt_num:  1440000  type:  <class 'int'>
==================== Youtube Raw Data Descriptions Ended ==================== 

==========inner loop==========

==================== Youtube Raw Data Descriptions ====================
title:  온라인꿈나무과학실험교실ㅣ교과서가 알려주지 않는 과학실험BEST5ㅣ경기도융합과학교육원
author:  융과원TV 경기도융합과학교육원
thumb_url:  https://i.ytimg.com/vi/QKXfI6KMeY0/maxresdefault.jpg
view_cnt:  3345  type:  <class 'int'>
duration:  1409  type:  <class 'int'>
youtube_category:  Education
youtube_category_id:  27
is_family_friendly:  true
video_id:  QKXfI6KMeY0
author_id:  UCln13PvB4fOdYGSoI2KlA4Q
yupload_time:  2020-08-24
keywords:  
like_dislike_cnt:  26 / 0
like_cnt:  26  type:  <class 'int'>
dislike_cnt:  0  type:  <class 'int'>
title_hash_tags_str:  

Exception:  'description'
==========inner loop==========

===============

,title,authorNick,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords,searchKeyword
